# Find duplicate questions on StackOverflow by their embeddings

calculate a similarity for pieces of text to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

We will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

File data/train.tsv is already downloaded.
File data/validation.tsv is already downloaded.
File data/test.tsv is already downloaded.
File data/test_embeddings.tsv is already downloaded.


Removed incomplete download


KeyboardInterrupt: 

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [77]:
from grader import Grader

In [78]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [preferred] [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. `GoogleNews-vectors-negative300.bin.gz` will be downloaded in `download_week3_resources()`.
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [79]:
import gensim

In [80]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(fname='/home/karen/Downloads/data/GoogleNews-vectors-negative300.bin.gz', binary=True)

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [81]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [82]:
print(check_embeddings(wv_embeddings))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


These embeddings look good.


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [83]:
import numpy as np

In [192]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    if len(question) == 0:
        return np.zeros(dim)
    # both unk and known words
    word_embed = np.array([embeddings[word] for word in question.split() if word in embeddings])
    if len(word_embed) == 0:
        # all unk words
         word_embed = np.zeros(dim)
    else:
        word_embed = np.mean(word_embed, axis=0)
    return word_embed

To check the basic correctness:

In [85]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [86]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [87]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /home/karen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
question2vec_result = []
i=0
for question in open('/home/karen/Downloads/data/stackoverflow_qa/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    if i<10:
        print(answer.shape)
        i+=1
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
Current answer for task Question2Vec is: 0.01929389126598835
-0.02872721292078495
0.0460561104118824
0.0852593332529068
0.0243055559694767
-0...


In [89]:
question2vec_result, question2vec_result.shape

(array([ 0.01929389, -0.02872721,  0.04605611, ..., -0.11884562,
        -0.01780192,  0.01442464]), (30000,))

Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. __The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.__

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [90]:
(np.array([2,3,4])<=1), (np.array([2,3,4])<=1).astype(int), (np.array([2,3,4])<=1).astype(int).sum()/len([2,3,4])

(array([False, False, False]), array([0, 0, 0]), 0.0)

In [91]:
(np.array([2,3,4])<=2), (np.array([2,3,4])<=2).astype(int), (np.array([2,3,4])<=2).astype(int).sum()/len([2,3,4])

(array([ True, False, False]), array([1, 0, 0]), 0.3333333333333333)

In [92]:
(np.array([2,3,4])<=3), (np.array([2,3,4])<=3).astype(int), (np.array([2,3,4])<=3).astype(int).sum()/len([2,3,4])

(array([ True,  True, False]), array([1, 1, 0]), 0.6666666666666666)

In [93]:
(np.array([2,3,4])<=4), (np.array([2,3,4])<=4).astype(int), (np.array([2,3,4])<=4).astype(int).sum()/len([2,3,4])

(array([ True,  True,  True]), array([1, 1, 1]), 1.0)

In [94]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one is 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    hit = 0
    boolean_check = (np.array(dup_ranks)<=k)
    ranks = boolean_check.astype(int)
    print('ranks', ranks)
    hit = ranks.sum()/len(dup_ranks)
    return hit

Test your code on the tiny examples:

In [95]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    print('dup_ranks', dup_ranks)
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        print('k', k, 'c', correct, 'hits_count', hits_count(dup_ranks, k))
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    print('dup_ranks', dup_ranks)
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        print('k', k, 'c', correct, 'hits_count', hits_count(dup_ranks, k))
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    print('dup_ranks', dup_ranks)
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        print('k', k, 'c', correct, 'hits_count', hits_count(dup_ranks, k))
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    print('dup_ranks', dup_ranks)
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        print('k', k, 'c', correct, 'hits_count', hits_count(dup_ranks, k))
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [96]:
print(test_hits())

dup_ranks [2]
ranks [0]
k 1 c 0 hits_count 0.0
ranks [0]
ranks [1]
k 2 c 1 hits_count 1.0
ranks [1]
ranks [1]
k 3 c 1 hits_count 1.0
ranks [1]
ranks [1]
k 4 c 1 hits_count 1.0
ranks [1]
dup_ranks [1, 1]
ranks [1 1]
k 1 c 1 hits_count 1.0
ranks [1 1]
ranks [1 1]
k 2 c 1 hits_count 1.0
ranks [1 1]
dup_ranks [2, 1]
ranks [0 1]
k 1 c 0.5 hits_count 0.5
ranks [0 1]
ranks [1 1]
k 2 c 1 hits_count 1.0
ranks [1 1]
dup_ranks [2, 2]
ranks [0 0]
k 1 c 0 hits_count 0.0
ranks [0 0]
ranks [1 1]
k 2 c 1 hits_count 1.0
ranks [1 1]
Basic test are passed.


In [97]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one is 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    return (1 / np.log2(1+np.array(dup_ranks)[np.array(dup_ranks)<=k])).sum()/len(dup_ranks)

In [98]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [99]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [100]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [101]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

ranks [1]
ranks [1 0]
ranks [1 1]
ranks [0 1]
ranks [1 1]
ranks [1 0 0]
ranks [1 1 0]
ranks [1 1 1]
ranks [1 0 0 0 0 0 0 0 0 0]
ranks [1 1 0 0 0 0 0 0 0 0]
ranks [1 1 1 0 0 0 0 0 0 0]
ranks [1 1 1 1 0 0 0 0 0 0]
ranks [1 1 1 1 1 0 0 0 0 0]
ranks [1 1 1 1 1 1 0 0 0 0]
ranks [1 1 1 1 1 1 1 0 0 0]
ranks [1 1 1 1 1 1 1 1 0 0]
ranks [1 1 1 1 1 1 1 1 1 0]
ranks [1 1 1 1 1 1 1 1 1 1]
ranks [0 0 0 0 0 0 0 0 1 0]
ranks [0 0 0 1 0 0 0 0 1 0]
ranks [0 0 0 1 0 0 0 0 1 1]
ranks [0 0 1 1 0 0 0 0 1 1]
ranks [0 1 1 1 0 0 0 0 1 1]
ranks [0 1 1 1 0 0 0 1 1 1]
ranks [0 1 1 1 0 0 1 1 1 1]
ranks [0 1 1 1 1 0 1 1 1 1]
ranks [1 1 1 1 1 0 1 1 1 1]
ranks [1 1 1 1 1 1 1 1 1 1]
ranks [0 0 0 1 0 0 0 0 0 0]
ranks [0 0 0 1 0 0 0 0 1 0]
ranks [0 1 0 1 0 0 0 0 1 0]
ranks [1 1 0 1 0 0 0 0 1 0]
ranks [1 1 1 1 0 0 0 0 1 0]
ranks [1 1 1 1 0 0 0 0 1 1]
ranks [1 1 1 1 0 0 1 0 1 1]
ranks [1 1 1 1 0 0 1 1 1 1]
ranks [1 1 1 1 1 0 1 1 1 1]
ranks [1 1 1 1 1 1 1 1 1 1]
ranks [0 1 0 0 0 0 0 0 0 0]
ranks [0 1 0 0 1 0 0 0 0 0]
rank

In [102]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [103]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [104]:
validation = read_corpus(filename='/home/karen/Downloads/data/stackoverflow_qa/validation.tsv')

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [201]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    
    question_e = [question_to_vec(question, embeddings, dim)]
    candidates_e = [question_to_vec(i, embeddings, dim) for i in candidates]
    # print(np.array(question_e).shape, np.array(candidates_e).shape)
    similarity = cosine_similarity(question_e,candidates_e)
    # sorted_candidates = sorted(dict(zip(similarity[0], candidates)).items(), reverse=True)
    # return [(candidates.index(s[1]), s[1]) for s in sorted_candidates]
    return [(index, candidates[index]) for index in np.argsort(similarity)[0][::-1]]

Test your code on the tiny examples:

In [107]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        print(ranks)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [108]:
print(test_rank_candidates())

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]
Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [109]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [110]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

ranks [0 0 0 ... 0 0 1]
DCG@   1: 0.214 | Hits@   1: 0.214
ranks [0 0 1 ... 0 0 1]
DCG@   5: 0.268 | Hits@   5: 0.316
ranks [1 0 1 ... 0 0 1]
DCG@  10: 0.285 | Hits@  10: 0.367
ranks [1 0 1 ... 0 0 1]
DCG@ 100: 0.323 | Hits@ 100: 0.557
ranks [1 0 1 ... 1 0 1]
DCG@ 500: 0.355 | Hits@ 500: 0.814
ranks [1 1 1 ... 1 1 1]
DCG@1000: 0.375 | Hits@1000: 1.000


In [111]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


In [133]:
import re
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = GOOD_SYMBOLS_RE.sub('', text)
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS])
    return text.strip()

In [165]:
prepared_validation = []
for line in validation:
    prepared_validation = [text_prepare(i) for i in line]
prepared_validation = np.array(prepared_validation)[:, np.newaxis].T

In [119]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [120]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

ranks [0 0 0 ... 0 0 0]
DCG@   1: 0.011 | Hits@   1: 0.011
ranks [0 0 0 ... 0 0 0]
DCG@   5: 0.061 | Hits@   5: 0.118
ranks [0 0 0 ... 0 0 0]
DCG@  10: 0.102 | Hits@  10: 0.247
ranks [1 1 1 ... 1 1 1]
DCG@ 100: 0.263 | Hits@ 100: 0.999
ranks [1 1 1 ... 1 1 1]
DCG@ 500: 0.263 | Hits@ 500: 1.000
ranks [1 1 1 ... 1 1 1]
DCG@1000: 0.263 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [131]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [132]:
prepare_file("/home/karen/Downloads/data/stackoverflow_qa/train.tsv","/home/karen/Downloads/data/stackoverflow_qa/train_prepared.tsv")
prepare_file("/home/karen/Downloads/data/stackoverflow_qa/test.tsv","/home/karen/Downloads/data/stackoverflow_qa/test_prepared.tsv")

In [123]:
test_prep  = read_corpus("/home/karen/Downloads/data/stackoverflow_qa/test_prepared.tsv")

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [124]:
def array_to_string(arr):
    return '\n'.join(str(num) for num in arr)

def matrix_to_string(matrix):
    return '\n'.join('\t'.join(str(num) for num in line) for line in matrix)

In [174]:
w2v_ranks_results = []
prepared_test_data = '/home/karen/Downloads/data/stackoverflow_qa/test_prepared.tsv'
i=0
for line in open(prepared_test_data):
    q, *candidates = line.strip().split('\t')
    ranks = rank_candidates(q, candidates, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    if i<3:
        print(ranks[:10])
        print([r[0] for r in ranks[:10]])
        print([ranked_candidates.index(i) + 1 for i in range(1, 10)])
        i+=1
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

(1, 300) (100, 300)
[(39, 'angularfire impossible recover time entered user database'), (22, 'get full description activity cleartool'), (67, 'importing another module another subdirectory current directorys parent directory python'), (60, 'minimize function selection function call overhead'), (99, 'doctrine preupdate invokes doesnt change related entity property prepersist'), (13, 'description item name paypal payment received email'), (2, 'development tools permission set'), (55, 'user input restriction goes haywire'), (3, 'go scene without using button flash asc3'), (74, 'problem jquery editinplace live function need ninja')]
[39, 22, 67, 60, 99, 13, 2, 55, 3, 74]
[94, 7, 9, 64, 37, 32, 93, 24, 100]
(1, 300) (100, 300)
[(8, 'notice compatible server client encryption algorithms found'), (6, 'authentication internal external users'), (15, 'choosing cloud storage service web api ftp third party server'), (30, 'use sorting algorithm java'), (43, 'aspnet core running windows service get

(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)


(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)
(1, 300) (100, 300)


KeyboardInterrupt: 

## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour. The size of the embeddings' dictionary should be approximately 100 000 (number of lines in the result file). If you got significantly more than this number, try to check all the instructions above.

In [186]:
!Starspace/starspace train -trainFile /home/karen/Downloads/data/stackoverflow_qa/train_prepared.tsv -trainMode 1 -adagrad  true -ngrams  1 -dim 100 -minCount  2 -verbose true -fileFormat labelDoc -negSearchLimit 10 -lr 0.05 -model starSpaceModel

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 1
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : /home/karen/Downloads/data/stackoverflow_qa/train_prepared.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : /home/karen/Downloads/data/stackoverflow_qa/train_prepared.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040210  loss: 0.041422  eta: 0h2m  tot: 0h0m43s  (20.0%)3%  lr: 0.049039  loss: 0.096336  eta: 0h3m  tot: 0h0m4s  (2.3%)15.8%  lr: 0.04

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [181]:
i=0
for vec in open('starSpaceModel.tsv'):
    if i<3:
        print(vec.split('\t')[0], list(map(lambda x: x.replace('\n', ''), vec.split('\t')[1:])))
        i+=1

using ['0.029723', '0.00126978', '0.0109264', '-0.00850125', '-0.0038752', '-0.0136108', '0.0143892', '0.021096', '0.0164289', '0.0155355', '-0.00481273', '0.00164163', '-0.0135173', '-0.022791', '-0.00448459', '-0.00856387', '0.0123847', '-0.0166097', '-0.0145041', '-0.00913096', '-0.00511055', '-0.00183105', '0.00316092', '0.00750908', '0.0140855', '-0.00503389', '-0.0233016', '-0.0110876', '-0.000840746', '-0.00260444', '0.00594024', '-0.00771097', '-0.0171782', '0.00662687', '-0.0187548', '-0.000361999', '0.001136', '0.0155511', '-0.00695704', '0.0100371', '-0.0239684', '-0.0160961', '0.0239641', '0.00748706', '-0.0272093', '-0.0109743', '-0.00453283', '0.0126349', '-0.0172669', '-0.0192649', '-0.0284429', '0.00128225', '-0.00354485', '0.0227757', '-0.0119172', '-0.0434495', '-0.00242183', '0.0132759', '0.00843015', '0.010946', '0.00359454', '-0.0123595', '-0.0323981', '-0.00386617', '0.0219424', '0.00188741', '-0.00872709', '-0.00303016', '0.00249568', '0.00170681', '-0.0189029', 

In [182]:
words = [vec.split('\t')[0] for vec in open('starSpaceModel.tsv')]
word_vec = [list(map(lambda x: float(x.replace('\n', '')), vec.split('\t')[1:])) for vec in open('starSpaceModel.tsv')]

In [183]:
words[:3], word_vec[:3]

(['using', 'android', 'file'],
 [[0.029723,
   0.00126978,
   0.0109264,
   -0.00850125,
   -0.0038752,
   -0.0136108,
   0.0143892,
   0.021096,
   0.0164289,
   0.0155355,
   -0.00481273,
   0.00164163,
   -0.0135173,
   -0.022791,
   -0.00448459,
   -0.00856387,
   0.0123847,
   -0.0166097,
   -0.0145041,
   -0.00913096,
   -0.00511055,
   -0.00183105,
   0.00316092,
   0.00750908,
   0.0140855,
   -0.00503389,
   -0.0233016,
   -0.0110876,
   -0.000840746,
   -0.00260444,
   0.00594024,
   -0.00771097,
   -0.0171782,
   0.00662687,
   -0.0187548,
   -0.000361999,
   0.001136,
   0.0155511,
   -0.00695704,
   0.0100371,
   -0.0239684,
   -0.0160961,
   0.0239641,
   0.00748706,
   -0.0272093,
   -0.0109743,
   -0.00453283,
   0.0126349,
   -0.0172669,
   -0.0192649,
   -0.0284429,
   0.00128225,
   -0.00354485,
   0.0227757,
   -0.0119172,
   -0.0434495,
   -0.00242183,
   0.0132759,
   0.00843015,
   0.010946,
   0.00359454,
   -0.0123595,
   -0.0323981,
   -0.00386617,
   0.021942

In [184]:
starspace_embeddings = {w:np.array(v) for w, v in zip(words,word_vec)}

In [185]:
starspace_embeddings["words"].shape

(100,)

In [186]:
i=0
for w,v in starspace_embeddings.items():
    if i<3:
        print(w, v)
        i+=1

using [ 0.029723    0.00126978  0.0109264  -0.00850125 -0.0038752  -0.0136108
  0.0143892   0.021096    0.0164289   0.0155355  -0.00481273  0.00164163
 -0.0135173  -0.022791   -0.00448459 -0.00856387  0.0123847  -0.0166097
 -0.0145041  -0.00913096 -0.00511055 -0.00183105  0.00316092  0.00750908
  0.0140855  -0.00503389 -0.0233016  -0.0110876  -0.00084075 -0.00260444
  0.00594024 -0.00771097 -0.0171782   0.00662687 -0.0187548  -0.000362
  0.001136    0.0155511  -0.00695704  0.0100371  -0.0239684  -0.0160961
  0.0239641   0.00748706 -0.0272093  -0.0109743  -0.00453283  0.0126349
 -0.0172669  -0.0192649  -0.0284429   0.00128225 -0.00354485  0.0227757
 -0.0119172  -0.0434495  -0.00242183  0.0132759   0.00843015  0.010946
  0.00359454 -0.0123595  -0.0323981  -0.00386617  0.0219424   0.00188741
 -0.00872709 -0.00303016  0.00249568  0.00170681 -0.0189029  -0.00333945
  0.00156616 -0.00864972  0.0237695  -0.0192529   0.0186781  -0.00780661
  0.00895939  0.0363481  -0.0168534   0.0199988   0.00

In [187]:
np.array(prepared_validation).shape

(1, 1001)

In [188]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

(1, 100) (1000, 100)


In [189]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

ranks [1]
DCG@   1: 1.000 | Hits@   1: 1.000
ranks [1]
DCG@   5: 1.000 | Hits@   5: 1.000
ranks [1]
DCG@  10: 1.000 | Hits@  10: 1.000
ranks [1]
DCG@ 100: 1.000 | Hits@ 100: 1.000
ranks [1]
DCG@ 500: 1.000 | Hits@ 500: 1.000
ranks [1]
DCG@1000: 1.000 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [195]:
sentences = ['loops replaced linq statement', 'get flex app load quicker', 'wix installer verify sufficient privileges start system services', 'timer thread pthread c', 'unable use variable defined beforegroup test method', 'finding sequences data frame r', 'calculate possible pairs items two lists', 'display data parse objects recycler view android', 'access windows kernel system calls directly', 'listview items repetition', 'excel functions showing converting excel csv php', 'entity framework query using contains mulitple options', 'spring security filters authentication spring web services', 'force thread update imageview immediately', 'install python3 custom path using chocolatey', 'break lines inside div tag', 'combine html php html shown view source nested correctly', 'animation inside custom alertdialog', 'hadoop cluster wrong permissions upgrade', 'display youtube video html page', 'bootstrap 3 select styling ie', 'paint app jquery mobile object method', 'django admin list page', 'semantic ui grid display full width column computer columns hidden', 'vbscript creating scheduled task', 'bash script start process wait random kill process restart', 'r oracle sql', 'convert widestring string unicode bytes', 'c++ cannot convert parameter 1 const char char', 'get http 500 error use python urllib2open', 'start notification intentservice', 'jpahibernate database restart problem', 'vertical horizontal data arrangement page crystal reports', 'haskell techniques mixed part structure part unmodified text parsing rewriting', 'node js post request body undefined', 'didchangeobject called nsfetchedresultscontroller', 'unique 4 digit random number c#', 'tcp ip possible read tcp udp data program sending remotely', 'hibernate postgres target lists 1664 entries', 'importing chartjs aurelia causes page errors bunldling', 'edited uiimagepickercontrollerh mistake getting errors project', 'returning expression ghc monad', 'wcf error cannot obtain metadata', 'ajax polling symfony process component', 'extract data cell excel', 'treetagger r', 'cannot use ortools google', 'get javascript prompt value protractor', 'c# aspnet 35 deserialize json get object string', 'avoid net native bugs', 'run play framework port9001 windows cmd play console', 'hook sharepoint 2007 feature application_start site', 'pulling geojson data leaflet ajax call', 'jekyll button url working', 'iterative bst insertion c++', 'suitable way scale rest api window azure handle thousands requests', 'b character stored string python', 'switching bits nibble int', 'building clusters based partnership', 'create form select making value option text db array', 'es6 generators example yield expression first next', 'ripple drawable effect appearing differently listview recyclerview', 'matplotlib objectoriented code display inline notebook', 'disable enableglobalmethodsecurity annotation', 'upload file codeigniter app dropbox', 'uppercasing variable makefile', 'config_hid_multitouch parameter values android kernel', 'stuck fixing query mistakes two commands wrote', 'create array mysql query php', 'meteor possible flush per row table', 'foundation 4unable overwrite regular expression validate password', 'make 3d model onclick events display website', 'validate checkbox aspnet c#', 'dav svn config server hosting multiple domains', 'npm error specified procedure could found', 'matlab error using assignin attempt add c static workspace', 'view fit canvas canvas scale', 'query working worked hours', 'possible generate xml sql dont know number levels', 'create app aweber', 'multidimensional arrays using range simultaneously set start stop step', 'xamarin mac 210 compatible exif library', 'change string int', 'implement serverside rate limiting perl web service', 'change add chart data series pythonpptx', 'solving profit algorithm', 'way get list users custom audience', 'jms transaction', 'precision recall multiclassmultilabel classification', 'mavenpluginapi comthoughtworksqdoxparserparseexception', 'visual diagram format gui event mapping', 'sql server query get details table single column multiple rows', 'oracle recursive query using start connected', 'pass json data nunjucks template', 'unable replace arabic characters apache poi ms word file', 'android jobscheduler jobservice network constraint start even device network connection', 'reflexive manytomany relationship cakephp', 'show timer ios lock screen', 'error windows phone emulator', 'produce sum column sql']

In [202]:
starspace_ranks_results = []
prepared_test_data = '/home/karen/Downloads/data/stackoverflow_qa/test_prepared.tsv'
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, dim=100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 80	57	78	12	3	81	37	63	61	47	49	10	58	19	85	68	17	87	55	98	77	93	23	75	90	62	73	94	29	1	65	27	91	59	...


Please, **don't remove** the file with these embeddings because you will need them in the final project.

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [203]:
STUDENT_EMAIL = 'karenye.psu@gmail.com'
STUDENT_TOKEN = 'nRV2nALRnk9Cczc8'
grader.status()

You want to submit these parts:
Task Question2Vec: 0.01929389126598835
-0.02872721292078495
0.0460561104118824
0.0852593332529068
0.0243055559694767
-0...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....
Task DCGScore: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...
Task W2VTokenizedRanks: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...
Task StarSpaceRanks: 80	57	78	12	3	81	37	63	61	47	49	10	58	19	85	68	17	87	55	98	77	93	23	75	90	62	73	94	29	1	65	27	91	59	...


If you want to submit these answers, run cell below

In [204]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
